# Análisis de Diferencia de Diferencias por Población

Este notebook analiza las variables DifDif desglosadas por **Categoria_PASO_2023**.

## Análisis realizados:

1. **Estadísticas descriptivas por categoría**: Media, n, DE
2. **Tests de significancia por categoría**: Test t de una muestra (H₀: DifDif = 0)
3. **Comparación entre categorías**: Test de Kruskal-Wallis
4. **Tablas Excel por población**: Resultados formateados

## Poblaciones analizadas:

- Left_Wing
- Progressivism
- Centre
- Moderate_Right_A
- Moderate_Right_B
- Right_Wing_Libertarian

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from scipy.stats import kruskal
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

## 1. Cargar Datos

In [ ]:
# Cargar df_Elecciones
Ruta_Datos = os.path.join(os.getcwd(), '..', 'Data', 'Procesados', 'df_Elecciones.xlsx')
df_Elecciones = pd.read_excel(Ruta_Datos)

print(f"✓ df_Elecciones cargado exitosamente")
print(f"  Dimensiones: {df_Elecciones.shape}")

In [ ]:
# Mapa de colores para las categorías
Mapa_Colores_Categorias = {
    'Left_Wing': '#f65058',              # Rojo
    'Progressivism': '#0078bf',          # Azul
    'Centre': '#009cdd',                 # Celeste
    'Moderate_Right_A': '#f7d117',       # Amarillo
    'Moderate_Right_B': '#f7d117',       # Amarillo
    'Right_Wing_Libertarian': '#753bbd'  # Morado
}

Etiquetas_Categorias = {
    'Left_Wing': 'Left Wing',
    'Progressivism': 'Progressivism',
    'Centre': 'Centre',
    'Moderate_Right_A': 'Moderate Right A',
    'Moderate_Right_B': 'Moderate Right B',
    'Right_Wing_Libertarian': 'Right Wing Libertarian'
}

## 2. Identificar Variables y Categorías

In [ ]:
# Identificar columnas DifDif
columnas_difdif_co = [col for col in df_Elecciones.columns if col.startswith('DifDif_CO')]
columnas_difdif_ct = [col for col in df_Elecciones.columns if col.startswith('DifDif_CT')]

# Categorías
categorias = df_Elecciones['Categoria_PASO_2023'].unique()
categorias_validas = ['Left_Wing', 'Progressivism', 'Centre', 
                      'Moderate_Right_A', 'Moderate_Right_B', 'Right_Wing_Libertarian']

print(f"Variables DifDif_CO: {len(columnas_difdif_co)}")
print(f"Variables DifDif_CT: {len(columnas_difdif_ct)}")
print(f"\nCategorías encontradas: {list(categorias)}")

## 3. Función para Calcular Estadísticas por Categoría

In [ ]:
def Calcular_Estadisticas_Por_Categoria(df, columnas_difdif, categorias):
    """
    Calcula estadísticas descriptivas y test t por categoría.
    """
    
    resultados = []
    
    for columna in columnas_difdif:
        # Extraer número de ítem
        num_item = columna.split('_')[-1]
        
        fila_resultado = {'Variable': columna, 'Item': num_item}
        
        # Para cada categoría
        for categoria in categorias:
            # Filtrar datos de esta categoría
            datos = df[df['Categoria_PASO_2023'] == categoria][columna].dropna()
            
            if len(datos) > 0:
                # Estadísticas
                media = datos.mean()
                n = len(datos)
                de = datos.std()
                
                # Test t
                if n > 1:
                    t_stat, p_valor = stats.ttest_1samp(datos, 0)
                    
                    if p_valor < 0.001:
                        sig = '***'
                    elif p_valor < 0.01:
                        sig = '**'
                    elif p_valor < 0.05:
                        sig = '*'
                    else:
                        sig = 'ns'
                else:
                    p_valor = np.nan
                    sig = '—'
                
                fila_resultado[f'{categoria}_Media'] = media
                fila_resultado[f'{categoria}_n'] = n
                fila_resultado[f'{categoria}_DE'] = de
                fila_resultado[f'{categoria}_p'] = p_valor
                fila_resultado[f'{categoria}_Sig'] = sig
            else:
                fila_resultado[f'{categoria}_Media'] = np.nan
                fila_resultado[f'{categoria}_n'] = 0
                fila_resultado[f'{categoria}_DE'] = np.nan
                fila_resultado[f'{categoria}_p'] = np.nan
                fila_resultado[f'{categoria}_Sig'] = '—'
        
        # Test de Kruskal-Wallis entre categorías
        grupos = [df[df['Categoria_PASO_2023'] == cat][columna].dropna() 
                  for cat in categorias]
        grupos_validos = [g for g in grupos if len(g) > 1]
        
        if len(grupos_validos) > 1:
            try:
                h_stat, p_kw = kruskal(*grupos_validos)
                fila_resultado['KW_p'] = p_kw
                if p_kw < 0.001:
                    fila_resultado['KW_Sig'] = '***'
                elif p_kw < 0.01:
                    fila_resultado['KW_Sig'] = '**'
                elif p_kw < 0.05:
                    fila_resultado['KW_Sig'] = '*'
                else:
                    fila_resultado['KW_Sig'] = 'ns'
            except:
                fila_resultado['KW_p'] = np.nan
                fila_resultado['KW_Sig'] = '—'
        else:
            fila_resultado['KW_p'] = np.nan
            fila_resultado['KW_Sig'] = '—'
        
        resultados.append(fila_resultado)
    
    return pd.DataFrame(resultados)

## 4. Calcular Estadísticas para DifDif_CO

In [ ]:
print("Calculando estadísticas por categoría para DifDif_CO...\n")

df_Stats_CO_Cat = Calcular_Estadisticas_Por_Categoria(
    df_Elecciones, 
    columnas_difdif_co, 
    categorias_validas
)

print(f"✓ Estadísticas calculadas para {len(df_Stats_CO_Cat)} variables DifDif_CO")
print(f"\nPrimeras columnas: {df_Stats_CO_Cat.columns[:5].tolist()}")

## 5. Calcular Estadísticas para DifDif_CT

In [ ]:
print("Calculando estadísticas por categoría para DifDif_CT...\n")

df_Stats_CT_Cat = Calcular_Estadisticas_Por_Categoria(
    df_Elecciones, 
    columnas_difdif_ct, 
    categorias_validas
)

print(f"✓ Estadísticas calculadas para {len(df_Stats_CT_Cat)} variables DifDif_CT")

## 6. Funciones Auxiliares para Excel

In [ ]:
def hex_to_rgb(hex_color):
    """Convierte color hexadecimal a formato RGB."""
    hex_color = hex_color.lstrip('#')
    return hex_color.upper()


def aplicar_borde_completo(cell):
    """Aplica bordes."""
    thin_border = Border(
        left=Side(style='thin', color='000000'),
        right=Side(style='thin', color='000000'),
        top=Side(style='thin', color='000000'),
        bottom=Side(style='thin', color='000000')
    )
    cell.border = thin_border


def obtener_color_significancia(texto_sig):
    """Color de fondo según significancia."""
    if texto_sig == '***':
        return '90EE90'
    elif texto_sig == '**':
        return 'B8F4B8'
    elif texto_sig == '*':
        return 'D4F4D4'
    elif texto_sig == 'ns':
        return 'FFE4E1'
    else:
        return 'FFFFFF'


def obtener_color_texto_significancia(texto_sig):
    """Color de texto según significancia."""
    if texto_sig == '***':
        return '006400'
    elif texto_sig == '**':
        return '228B22'
    elif texto_sig == '*':
        return '000000'
    elif texto_sig == 'ns':
        return '8B0000'
    else:
        return '000000'

## 7. Función para Crear Tabla Excel por Población

In [ ]:
def Crear_Tabla_Excel_DifDif_Por_Poblacion(
    df_stats,
    tipo,
    categorias,
    nombre_archivo,
    carpeta_destino='Tablas_Excel_DifDif'
):
    """
    Crea tabla Excel con estadísticas por población.
    """
    
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    wb = Workbook()
    ws = wb.active
    ws.title = f'DifDif_{tipo}_Poblacion'
    
    fila = 1
    
    # TÍTULO
    num_cols = 2 + (len(categorias) * 4) + 2  # Ítem + (Cat*4) + KW
    ws.merge_cells(start_row=fila, start_column=1, end_row=fila, end_column=num_cols)
    cell_titulo = ws.cell(fila, 1)
    cell_titulo.value = f'Análisis por Población - Diferencia de Diferencias ({tipo})'
    cell_titulo.font = Font(bold=True, size=14, color='FFFFFF')
    cell_titulo.fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
    cell_titulo.alignment = Alignment(horizontal='center', vertical='center')
    
    fila += 2
    
    # ENCABEZADO NIVEL 1 - Categorías
    col = 1
    ws.cell(fila, col).value = ''
    ws.cell(fila, col).fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
    col += 1
    
    for categoria in categorias:
        etiqueta = Etiquetas_Categorias.get(categoria, categoria)
        color_cat = hex_to_rgb(Mapa_Colores_Categorias.get(categoria, '#2C5282'))
        
        # Combinar 4 columnas para cada categoría
        ws.merge_cells(start_row=fila, start_column=col, end_row=fila, end_column=col+3)
        cell = ws.cell(fila, col)
        cell.value = etiqueta
        cell.font = Font(bold=True, color='FFFFFF', size=11)
        cell.fill = PatternFill(start_color=color_cat, end_color=color_cat, fill_type='solid')
        cell.alignment = Alignment(horizontal='center', vertical='center')
        
        col += 4
    
    # Kruskal-Wallis
    ws.merge_cells(start_row=fila, start_column=col, end_row=fila, end_column=col+1)
    cell = ws.cell(fila, col)
    cell.value = 'K-W'
    cell.font = Font(bold=True, color='FFFFFF', size=11)
    cell.fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
    cell.alignment = Alignment(horizontal='center', vertical='center')
    
    fila += 1
    
    # ENCABEZADO NIVEL 2 - Métricas
    col = 1
    cell = ws.cell(fila, col)
    cell.value = 'Ítem'
    cell.font = Font(bold=True, color='FFFFFF', size=10)
    cell.fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
    cell.alignment = Alignment(horizontal='center', vertical='center')
    col += 1
    
    for _ in categorias:
        for header in ['Media', 'n', 'DE', 'Sig']:
            cell = ws.cell(fila, col)
            cell.value = header
            cell.font = Font(bold=True, color='FFFFFF', size=9)
            cell.fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
            cell.alignment = Alignment(horizontal='center', vertical='center')
            col += 1
    
    # Headers KW
    for header in ['p', 'Sig']:
        cell = ws.cell(fila, col)
        cell.value = header
        cell.font = Font(bold=True, color='FFFFFF', size=9)
        cell.fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
        cell.alignment = Alignment(horizontal='center', vertical='center')
        col += 1
    
    fila += 1
    
    # DATOS
    for idx, row in df_stats.iterrows():
        col = 1
        
        # Ítem
        cell = ws.cell(fila, col)
        cell.value = f"Ítem {row['Item']}"
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.fill = PatternFill(start_color='E7E6E6', end_color='E7E6E6', fill_type='solid')
        cell.font = Font(bold=True)
        col += 1
        
        # Para cada categoría
        for categoria in categorias:
            # Media
            cell = ws.cell(fila, col)
            media = row[f'{categoria}_Media']
            if not np.isnan(media):
                cell.value = f"{media:.3f}"
            else:
                cell.value = '—'
            cell.alignment = Alignment(horizontal='center', vertical='center')
            col += 1
            
            # n
            cell = ws.cell(fila, col)
            n = row[f'{categoria}_n']
            cell.value = int(n) if n > 0 else '—'
            cell.alignment = Alignment(horizontal='center', vertical='center')
            col += 1
            
            # DE
            cell = ws.cell(fila, col)
            de = row[f'{categoria}_DE']
            if not np.isnan(de):
                cell.value = f"{de:.3f}"
            else:
                cell.value = '—'
            cell.alignment = Alignment(horizontal='center', vertical='center')
            col += 1
            
            # Sig
            cell = ws.cell(fila, col)
            sig = row[f'{categoria}_Sig']
            cell.value = sig
            cell.alignment = Alignment(horizontal='center', vertical='center')
            color_fondo = obtener_color_significancia(sig)
            color_texto = obtener_color_texto_significancia(sig)
            cell.fill = PatternFill(start_color=color_fondo, end_color=color_fondo, fill_type='solid')
            cell.font = Font(bold=True, color=color_texto)
            col += 1
        
        # KW p-valor
        cell = ws.cell(fila, col)
        p_kw = row['KW_p']
        if not np.isnan(p_kw):
            if p_kw < 0.001:
                cell.value = '<0.001'
            else:
                cell.value = f"{p_kw:.4f}"
        else:
            cell.value = '—'
        cell.alignment = Alignment(horizontal='center', vertical='center')
        col += 1
        
        # KW Sig
        cell = ws.cell(fila, col)
        sig_kw = row['KW_Sig']
        cell.value = sig_kw
        cell.alignment = Alignment(horizontal='center', vertical='center')
        color_fondo = obtener_color_significancia(sig_kw)
        color_texto = obtener_color_texto_significancia(sig_kw)
        cell.fill = PatternFill(start_color=color_fondo, end_color=color_fondo, fill_type='solid')
        cell.font = Font(bold=True, color=color_texto)
        
        fila += 1
    
    # Aplicar bordes
    for row in ws.iter_rows(min_row=1, max_row=fila-1, min_col=1, max_col=num_cols):
        for cell in row:
            aplicar_borde_completo(cell)
    
    # Ajustar anchos
    ws.column_dimensions['A'].width = 12
    for i in range(2, num_cols + 1):
        col_letter = get_column_letter(i)
        ws.column_dimensions[col_letter].width = 10
    
    # Guardar
    ruta_completa = os.path.join(carpeta_destino, nombre_archivo)
    wb.save(ruta_completa)
    
    print(f"✅ Tabla Excel guardada en: {ruta_completa}")
    
    return ruta_completa

## 8. Generar Tablas Excel por Población

In [ ]:
# Tabla para DifDif_CO
print("Generando tabla Excel por población para DifDif_CO...\n")
Crear_Tabla_Excel_DifDif_Por_Poblacion(
    df_stats=df_Stats_CO_Cat,
    tipo='CO',
    categorias=categorias_validas,
    nombre_archivo='Tabla_DifDif_CO_Por_Poblacion.xlsx'
)

# Tabla para DifDif_CT
print("\nGenerando tabla Excel por población para DifDif_CT...\n")
Crear_Tabla_Excel_DifDif_Por_Poblacion(
    df_stats=df_Stats_CT_Cat,
    tipo='CT',
    categorias=categorias_validas,
    nombre_archivo='Tabla_DifDif_CT_Por_Poblacion.xlsx'
)

print("\n✅ Todas las tablas por población generadas exitosamente")

## 9. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN DEL ANÁLISIS POR POBLACIÓN")
print("="*70)

print(f"\n📊 Variables analizadas:")
print(f"   - DifDif_CO: {len(df_Stats_CO_Cat)} variables")
print(f"   - DifDif_CT: {len(df_Stats_CT_Cat)} variables")

print(f"\n👥 Poblaciones analizadas: {len(categorias_validas)}")
for cat in categorias_validas:
    n_total = len(df_Elecciones[df_Elecciones['Categoria_PASO_2023'] == cat])
    print(f"   - {Etiquetas_Categorias[cat]}: {n_total} personas")

print(f"\n📈 Diferencias entre poblaciones (Kruskal-Wallis):")
sig_kw_co = df_Stats_CO_Cat[df_Stats_CO_Cat['KW_Sig'].isin(['*', '**', '***'])]
sig_kw_ct = df_Stats_CT_Cat[df_Stats_CT_Cat['KW_Sig'].isin(['*', '**', '***'])]
print(f"   - DifDif_CO: {len(sig_kw_co)} de {len(df_Stats_CO_Cat)} variables muestran diferencias")
print(f"   - DifDif_CT: {len(sig_kw_ct)} de {len(df_Stats_CT_Cat)} variables muestran diferencias")

print(f"\n📁 Archivos Excel generados:")
print(f"   - Tabla_DifDif_CO_Por_Poblacion.xlsx")
print(f"   - Tabla_DifDif_CT_Por_Poblacion.xlsx")

print(f"\n🎯 Interpretación:")
print(f"   - Columnas por categoría: Media, n, DE, Sig (test t vs 0)")
print(f"   - K-W: Test de Kruskal-Wallis (diferencias entre poblaciones)")
print(f"   - K-W significativo → Las poblaciones difieren en DifDif")

print("\n" + "="*70)